https://support.zemax.com/hc/en-us/articles/4406838460819-Modelling-of-a-Shack-Hartmann-Sensor-for-eye-aberration-evaluation  
https://support.zemax.com/hc/en-us/articles/1500005575422-How-to-model-a-black-box-optical-system-using-Zernike-coefficients


In [39]:
import os 
import sys
import h5py
path = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
sys.path.insert(0,path)

import numpy as np
import matplotlib.pyplot as plt
import LightPipes as lp
from skimage.restoration import unwrap_phase
from utils.api_functions import *
from utils.opt_functions import *
from shack_hartmann.sh_functions import *



%run -i "\\alfs1.physics.ox.ac.uk\al\howards\Zemax\ZOS-API Projects\PythonZOSConnection\PythonZOSConnection.py"
fast_system(TheSystem) # this speeds up the program.

Found OpticStudio at:   %sc:\program files\zemax opticstudio
Connected to OpticStudio
Serial #:  L121911


<Figure size 432x288 with 0 Axes>

First I create a file that I will manually upload to zemax. This sets n_zernike

In [41]:
n_zernike = 45
zernike_surf = TheSystem.LDE.GetSurfaceAt(1)
zernike_surf.SurfaceData.NumberOfTerms = n_zernike
zernike_surf.SurfaceData.NormRadius = 7
# generate_zernike_file(zernike_coeffs,radius = 5) #here i create a file that can be read into zemax.

In [47]:
weights = np.flip(np.arange(1,45,1))**5
p = weights/np.sum(weights)
Nx = 512
N=Nx
chunks = 200
chunksize = 100

In [57]:

for a in range(chunks):
    print(str(a/chunks))
    images = np.zeros([chunksize,N,N])
    zernikes = np.zeros([chunksize,n_zernike])

    for i in range(chunksize):
        
        # Generate abberations
        main_aberration = np.random.choice(np.arange(1,45), p = p)
        # design aberrations not enrirely random but to lie around a main aberration
        zernike_coeffs = np.random.rand(n_zernike)/(1+np.abs(np.arange(0,n_zernike,1)-main_aberration))**(2*np.random.rand(1))

        set_zernikes(ZOSAPI, zernike_surf, n_zernike,zernike_coeffs)

        images[i] = get_spots(ZOSAPI,TheSystem,Nx) # get spot pattern
        zernikes[i] = zernike_coeffs
        
    data = {'spots' : images,
            'zernikes' : zernikes}

    if a == 0:
        #initialize
        h5f = h5py.File(('zemax_sh_monster.h5'), 'w')
            

        h5f.create_dataset('zernikes', data=data['zernikes'], compression="gzip", chunks=True, maxshape=(None,n_zernike))
        h5f.create_dataset('spots', data=data['spots'], compression="gzip", chunks=True, maxshape=(None,N,N))
    else:
        
        h5f['zernikes'].resize((h5f['zernikes'].shape[0] + data['zernikes'].shape[0]),axis=0)
        h5f['zernikes'][-data['zernikes'].shape[0]:] = data['zernikes']
        
        h5f['spots'].resize((h5f['spots'].shape[0] + data['spots'].shape[0]),axis=0)
        h5f['spots'][-data['spots'].shape[0]:] = data['spots']



h5f.close()


0.0
0.005
0.01
0.015
0.02
0.025
0.03
0.035
0.04
0.045
0.05
0.055
0.06
0.065
0.07
0.075
0.08
0.085
0.09
0.095
0.1
0.105
0.11
0.115
0.12
0.125
0.13
0.135
0.14
0.145
0.15
0.155
0.16
0.165
0.17
0.175
0.18
0.185
0.19
0.195
0.2
0.205
0.21
0.215
0.22
0.225
0.23
0.235
0.24
0.245
0.25
0.255
0.26
0.265
0.27
0.275
0.28
0.285
0.29
0.295
0.3
0.305
0.31
0.315
0.32
0.325
0.33
0.335
0.34
0.345
0.35
0.355
0.36
0.365
0.37
0.375
0.38
0.385
0.39
0.395
0.4
0.405
0.41
0.415
0.42
0.425
0.43
0.435
0.44
0.445
0.45
0.455
0.46
0.465
0.47
0.475
0.48
0.485
0.49
0.495
0.5
0.505
0.51
0.515
0.52
0.525
0.53
0.535
0.54
0.545
0.55
0.555
0.56
0.565
0.57
0.575
0.58
0.585
0.59
0.595
0.6
0.605
0.61
0.615
0.62
0.625
0.63
0.635
0.64
0.645
0.65
0.655
0.66
0.665
0.67
0.675
0.68
0.685
0.69
0.695
0.7
0.705
0.71
0.715
0.72
0.725
0.73
0.735
0.74
0.745
0.75
0.755
0.76
0.765
0.77
0.775
0.78
0.785
0.79
0.795
0.8
0.805
0.81
0.815
0.82
0.825
0.83
0.835
0.84
0.845
0.85
0.855
0.86
0.865
0.87
0.875
0.88
0.885
0.89
0.895
0.9
0.905
0.91
0.91